## 1. XOR 데이터 생성 (NumPy 사용)

In [ ]:
# [1] 라이브러리 import
import numpy as np
import torch
import torch.optim as optim

In [ ]:
# [2] XOR 데이터 정의
x_seeds = np.array([[0, 0], [1, 0], [0, 1], [1, 1]], dtype=np.float32)
y_seeds = np.array([0, 1, 1, 0])  # XOR 정답

In [ ]:
# [3] 1000개의 샘플 랜덤 생성 (복원 추출)
N = 1000
idxs = np.random.randint(0, 4, N)  # 0~3 인덱스 중 하나 랜덤 선택

In [ ]:
# [4] 샘플 복제
X = x_seeds[idxs]  # shape: (1000, 2)
Y = y_seeds[idxs]  # shape: (1000,)

In [ ]:
# [5] 입력 데이터에 노이즈 추가
X += np.random.normal(scale=0.25, size=X.shape)

# 노이즈 사용 이유 : 모델이 더 일반화된 학습을 하게 되고, 단순히 "외우는 것"이 아니라 패턴을 이해하는 것에 더 가까워진다.

## 2. Shallow Neural Network (PyTorch 사용)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class shallow_neural_network(nn.Module):
    def __init__(self, num_input_features, num_hiddens):
        super().__init__()
        self.num_input_features = num_input_features
        self.num_hiddens = num_hiddens

        # [PyTorch] 레이어 정의
        self.linear1 = nn.Linear(num_input_features, num_hiddens)  # 입력 → 은닉층
        self.linear2 = nn.Linear(num_hiddens, 1)                   # 은닉층 → 출력

        # [PyTorch] 비선형 활성 함수 정의
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        z1 = self.linear1(x)        # 선형 변환
        a1 = self.tanh(z1)          # 비선형 활성화
        z2 = self.linear2(a1)
        a2 = self.sigmoid(z2)       # 출력 확률화
        return a2

## SGD

In [ ]:
# 하이퍼파라미터 설정
num_epochs = 100
lr = 1.0
num_hiddens = 3

# 모델, 옵티마이저, 손실 함수 정의
model = shallow_neural_network(2, num_hiddens)
optimizer = optim.SGD(model.parameters(), lr=lr)
loss_fn = nn.BCELoss()  # Binary Cross Entropy Loss

In [ ]:
# 학습 루프
for epoch in range(num_epochs):
    cost = 0.0

    for x, y in zip(X, Y):
        x_torch = torch.from_numpy(x)               # 입력을 텐서로 변환
        y_torch = torch.FloatTensor([y])            # 정답도 텐서로 변환

        y_hat = model(x_torch)                      # 예측 수행

        loss = loss_fn(y_hat, y_torch)              # 손실 계산
        optimizer.zero_grad()
        loss.backward()                             # 전체 배치에 대해 역전파
        optimizer.step()                            # 파라미터 업데이트

        cost += loss.item()                          # 비용 누적

    cost = cost / len(X)                             # 평균 손실

    if epoch % 10 == 0:
        print(epoch, cost)                           # 출력

0 0.6490769769730396
10 0.40723718070657744
20 0.4217062960174198
30 0.4116158128803936
40 0.40428774995582534
50 0.400452262459874
60 0.39858843055778836
70 0.3940070871035341
80 0.3995615531542262
90 0.4044992880507343


In [ ]:
for x, y in zip(x_seeds, y_seeds):
    print(x)  # 입력 벡터 출력

    x_torch = torch.FloatTensor(x)       # 입력 벡터를 텐서로 변환
    y_hat = model(x_torch)               # 모델 예측 수행

    print(y, y_hat.item())               # 정답 y와 예측값 ŷ 출력


[0. 0.]
0 0.00021113091497682035
[1. 0.]
1 0.9409389495849609
[0. 1.]
1 0.9977951049804688
[1. 1.]
0 0.07199577242136002


## GD

In [ ]:
# 하이퍼파라미터 설정
num_epochs = 100
lr = 1.0
num_hiddens = 3

# 모델, 옵티마이저, 손실 함수 정의
model = shallow_neural_network(2, num_hiddens)
optimizer = optim.SGD(model.parameters(), lr=lr)
loss = nn.BCELoss()  # Binary Cross Entropy Loss

In [ ]:
# 학습 루프
for epoch in range(num_epochs):
    optimizer.zero_grad()

    cost = 0.0

    for x, y in zip(X, Y):
        x_torch = torch.from_numpy(x)               # 입력을 텐서로 변환
        y_torch = torch.FloatTensor([y])            # 정답도 텐서로 변환

        y_hat = model(x_torch)                      # 예측 수행

        loss_val = loss(y_hat, y_torch)             # 손실 계산
        cost += loss_val                            # 비용 누적

    cost = cost / len(X)                             # 평균 손실
    cost.backward()                                  # 전체 배치에 대해 역전파
    optimizer.step()                                 # 파라미터 업데이트

    if epoch % 10 == 0:
        print(epoch, cost)                    # 출력 시 .item()으로 숫자화

0 tensor(0.7506, grad_fn=<DivBackward0>)
10 tensor(0.6907, grad_fn=<DivBackward0>)
20 tensor(0.6838, grad_fn=<DivBackward0>)
30 tensor(0.6722, grad_fn=<DivBackward0>)
40 tensor(0.6502, grad_fn=<DivBackward0>)
50 tensor(0.6118, grad_fn=<DivBackward0>)
60 tensor(0.5527, grad_fn=<DivBackward0>)
70 tensor(0.4744, grad_fn=<DivBackward0>)
80 tensor(0.3943, grad_fn=<DivBackward0>)
90 tensor(0.3317, grad_fn=<DivBackward0>)


In [ ]:
for x, y in zip(x_seeds, y_seeds):
    print(x)  # 입력 벡터 출력

    x_torch = torch.FloatTensor(x)       # 입력 벡터를 텐서로 변환
    y_hat = model(x_torch)               # 모델 예측 수행

    print(y, y_hat.item())               # 정답 y와 예측값 ŷ 출력


[0. 0.]
0 0.05986512452363968
[1. 0.]
1 0.8095449805259705
[0. 1.]
1 0.8704604506492615
[1. 1.]
0 0.2041512280702591
